<a href='http://www.holoviews.org'><img src="assets/header_logo.png" alt="HoloViews logo" width="20%;" align="left"/></a>
<div style="float:right;"><h2>01. Introduction to Elements</h2></div>

## Preliminaries

If the  ``hvtutorial`` environment has been correctly created and activated using the instructions listed on the [index](00-index.ipynb) page, the following imports should run and ``hv.extension('bokeh')``  should present a small HoloViews logo:

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
hv.extension('bokeh')

Here we import numpy and pandas with their standard aliases and HoloViews with its standard short form of ``hv``. The line reading ``hv.extension('bokeh')`` loads and activates the bokeh plotting backend so all visualizations will be generated using [Bokeh](bokeh.pydata.org). We will see how to use matplotlib instead of bokeh later in the tutorial [Customizing Visual Appearance](./02-customizing-visual-appearance.ipynb).

# What are elements?

In short, elements are HoloViews' most basic, core primitives that are all subclasses of ``Element``. All ``Element`` objects accept semantic metadata that allows their input data to be given an automatic, visual representation. Most importantly, element objects always preserve the raw data they are supplied.

In this notebook we will explore a number of different element types and examine some of the ways that elements can supplement the supplied data with useful semantic data. There are many element types you might chose in the exercises and to help you make your choice, you can browse the [visual reference](http://holoviews.org/reference/index.html) which shows all the element types.

## Creating elements

All basic elements accept their data as a single, mandatory positional argument which may be supplied in a number of different formats, some of which we will now examine. A handful of *annotation* elements are exceptions to this rule, namely ``Arrow``, ``Text``, ``Bounds``, ``Box`` and ``Ellipse`` as they require additional positional arguments.

### A simple curve

To start with a simple example, here is a [``Curve``](http://build.holoviews.org/reference/elements/bokeh/Curve.html) which we will use to plot the quadratic function:

In [ ]:
xs = [i for i in range(-10,11)]
ys = [100-(i**2) for i in range(-10,11)]
simple_curve = hv.Curve((xs,ys))
simple_curve

Here we supplied two lists in a tuple to [``hv.Curve``]((http://build.holoviews.org/reference/elements/bokeh/Curve.html), resulting in the curve object above. What you are looking at is the rich visual representations of this object generated by HoloViews using Bokeh. To demonstrate that this is an object and not a plot (which is carried out behind the scenes), here is the textual representation of this ``curve``:

In [ ]:
print(simple_curve)

There are a number of similar elements to [``Curve``](http://build.holoviews.org/reference/elements/bokeh/Curve.html) such as [``Area``](http://build.holoviews.org/reference/elements/bokeh/Area.html) and [``Scatter``](http://build.holoviews.org/reference/elements/bokeh/Scatter.html) which you can try out for yourself in the exercises.

In [ ]:
# Exercise: Try switching hv.Curve with hv.Area and hv.Scatter

In [ ]:
# Optional: 
# Look at the .data attribute of the elements you created to see the raw data (as a pandas DataFrame)

### Annotating the curve

So far the only thing that we have done with our two lists (``xs`` and ``ys``) is wrap it in different types of elements. There is more to annotating your data with suitable semantics that just declaring the type, for instance we might want to specify what the x-axis and y-axis actual correpond to. Perhaps this parabola is the trajectory of a thrown ball:

In [ ]:
trajectory = hv.Curve((xs,ys), kdims=['distance'], vdims=['height'])
trajectory

We have added *semantic* information about our curve to the [``Curve``](http://build.holoviews.org/reference/elements/bokeh/Curve.html) element where the ``kdim`` or *key dimension* corresponds to the independent variable ('distance') and the ``vdim`` or *value dimension* is the dependent variable. This additional information is *about* the data and is reflected in it's visual representation: we see that the axes above have now been labelled accordingly.

### Casting between elements

The type of an element tells you something about the structure of the data and gives HoloViews the appropriate hint required to generate a suitable visual representation from it. That said, the nature of the dimensions are actual *more* fundamental to the essential semantics than the type which is why HoloViews makes it trivial to cast between compatible element types:

In [ ]:
hv.Scatter(simple_curve)

The ability to cast between elements is very useful and can be a lot easier than having to declare them anew. Note how the x- and y-axes continues to correspond to distance and height respectively as the element types change. In other words, once you have declared the key and value dimensions, this information propagated appropriately to the derived elements.

In [ ]:
# Try casting the scatter shown above to an area then back to a curve.

### Turning arrays into elements

The curve above was constructed from a list of x-values and a list of y-values. Next we will create an element using an entirely different datatype, namely a [NumPy](http://www.numpy.org/) array:

In [ ]:
x = np.linspace(0, 10, 500)
y = np.linspace(0, 10, 500)
xx, yy = np.meshgrid(x, y)
arr = np.sin(xx)*np.cos(yy)

This two dimensional array can be pass to [``Image``]((http://build.holoviews.org/reference/elements/bokeh/Image.html) to create an image element:

In [ ]:
hv.Image(arr)

In [ ]:
# Exercise: Try visualizing different arrays.
# You can try a new function entirely or simple modifications of the existing one
# e.g explore the effect of squaring and cubing the sine and cosine terms

In [ ]:
# Optional: Try supplying appropriate labels for the x- and y- axes
# Hint: The x,y positions are how you *index* (or key) the array *values* (so x and y are both kdims)

### Turning tables into elements

In addition to simple Python literals and array types (including [NumPy](www.numpy.org) and  [xarray](xarray.pydata.org)), HoloViews elements can be passed tabular data in the form of pandas [DataFrames](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html):

In [ ]:
economic_data = pd.read_csv('../data/economic.csv')
economic_data.head()

Let's build an element that helps us understand how the percentage growth in US GDP varies over time. As our dataframe contains GDP growth data for lots of countries, let us select the United States from the table and create a [``Curve``](http://build.holoviews.org/reference/elements/bokeh/Curve.html) element from it:

In [ ]:
US_data = economic_data[economic_data['country'] == 'United States'] # Select data for the US only
US_data.head()

In [ ]:
growth = hv.Curve(US_data, kdims=['year'], vdims=['growth'])
growth

Here we see that the ``kdims`` and ``vdims`` are not simply axis labels: the supplied ``DataFrame`` already has dimensions names as column names. Instead, what the ``kdims`` and ``vdims`` are doing is selecting which of our columns should be along the x-axis and which should be along the y-axis.

In [ ]:
# Exercise: Plot the unemployment (unem) over year

### Dimension labels

In this example, the simplistic axis labels are starting to getting rather limiting. Changing the ``kdims`` and ``vdims`` is no longer trivial either, as they need to match the column names in the dataframe. Is the only solution to rename the columns in our dataframe?

*Absolutely not!* The recommendation is that you use short, programmer and pandas-friendly, [tab-completeable](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe-column-attribute-access-and-ipython-completion) column names as these are also the best dimension names to use with HoloViews.

What you should do instead is set the *dimension labels* as dimensions real rich objects behind the scenes: 

In [ ]:
gdp_growth = growth.redim.label(growth='GDP growth')
gdp_growth

With the ``redim`` method, we have associated a *dimension label* with ``growth``, resulting in a new object called ``gdp_growth`` (you can check for yourself that ``growth`` is unchanged). Let's look at what the value dimension really looks like:

In [ ]:
gdp_growth.vdims

In [ ]:
# Exercise: Use redim.label to give the year dimension a better label

The ``redim`` utility lets you easily change other [dimension parameters]() so let's give our GDP growth dimension the appropriate unit:

In [ ]:
gdp_growth.redim.unit(growth='%')

In [ ]:
# Exercise: Use redim.unit to give the year dimension a better unit 
# For instance, relabel to 'Time' then give the unit as 'year'

## Composing elements together

Viewing a single element at a time often conveys very little information for the space used. In this section, we introduce the two composition operators ``+`` and ``*`` to build [``Layout``](http://build.holoviews.org/reference/containers/bokeh/Layout.html) and [``Overlay``](http://build.holoviews.org/reference/containers/bokeh/Overlay.html) objects.

### Layouts

Earlier on we were casting a parabola to different element types. Viewing the different types was awkward, wasting lots of vertical space in the notebook. What we really want is to view these elements side-by-side:

In [ ]:
layout = trajectory + hv.Scatter(trajectory) + hv.Area(trajectory) + hv.Spikes(trajectory)
layout.cols(2)

What we have created with the ``+`` operator is a [``Layout``](http://build.holoviews.org/reference/containers/bokeh/Layout.html) object and we have supplied the hint that a two-column layout is desired:

In [ ]:
print(layout)

Now let us build a new layout by selecting elements from ``layout``:

In [ ]:
layout.Curve.I + layout.Spikes.I

We see that a ``Layout`` lets us pick component elements via two levels of tab-completable attribute access. Note that by default the type of the element defines the first level of access and the second level of access automatically uses roman numerals (as Python identifiers cannot start with numbers).

These two levels correspond to another type of annotation that applies to the elements directly (rather than their dimensions) called ``group`` and ``label``:

In [ ]:
cannonball = trajectory.relabel('Cannonball', group='Trajectory')
integral = hv.Area(trajectory).relabel('Filled', group='Trajectory')
labelled_layout = cannonball + integral
labelled_layout 

In [ ]:
# Exercise: Try out the tab-completion of labelled_layout to build a new layout swapping the position of these elements

In [ ]:
# Optional: Try using two levels of dictionary-style access to grab the cannonball trajectory

### Overlays

You can put (almost!) any two HoloViews objects into a Layout but there is a second ``*`` operator which is useful when two elements lives in the same space (i.e matching dimensions with similar ranges) to create an [``Overlay``](http://build.holoviews.org/reference/containers/bokeh/Overlay.html):

In [ ]:
trajectory * hv.Spikes(trajectory)

The indexing system of [``Overlay``](http://build.holoviews.org/reference/containers/bokeh/Overlay.html) is identical to that of [``Layout``](http://build.holoviews.org/reference/containers/bokeh/Layout.html).

In [ ]:
# Exercise: Make an overlay of the Spikes object from layout on top of the filled trajectory area of labelled_layout

In [ ]:
labelled_layout.Trajectory.Filled * layout.Spikes.I

One thing that is specify to Overlays is the use of color cycles to automatically differentiate between elements of the same type and ``group``:

In [ ]:
tennis_ball = cannonball.clone((xs, 0.5*np.array(ys)), label='Tennis ball')
cannonball + tennis_ball + (cannonball * tennis_ball)

Here we use the ``clone`` method to make a shallower tennis ball trajectory: the ``clone`` method create a new object that preserves semantic metadata while allowing overrides (in this case we override the input data and the ``label``).

Note that in the overlay, these two 'trajectory' curves are distinguished  from each other with a color cycle where the legend uses the assigned labels.

In [ ]:
# Optional Exercise: 
# 1. Create a thrown_ball curve with half the height of tennis_ball by cloning it and assigning the label 'Thrown ball'
# 2. Add thrown_ball to the overlay

# Onwards

Later in the tutorial, we will see how elements and the principles of composition extend to *containers* (such as ) which make data exploration quick, easy and interactive. Before we examine the container types, we will look at how to customize the appearance of elements, change the plotting extension and specify output formats.

For a quick preview of what we will be covering, hit the kernel restart button (⟳) in the notebook toolbar, change ``hv.extension('bokeh')`` to ``hv.extension('matplotlib')`` in the first cell and rerun the notebook!